In [1]:
import json
import spacy
from collections import Counter
import Levenshtein
import re
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

In [2]:
def custom_tokenizer(nlp):
    infixes = nlp.Defaults.infixes
    # Suppression de la division “-” nouvelle expression regex
    infixes = [pattern for pattern in infixes if '-' not in pattern]
    infix_re = compile_infix_regex(infixes)
    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                     suffix_search=nlp.tokenizer.suffix_search,
                     infix_finditer=infix_re.finditer,
                     token_match=nlp.tokenizer.token_match)

In [3]:
# Chargement du modèle linguistique polonais
nlp = spacy.load("pl_core_news_sm")
nlp.tokenizer = custom_tokenizer(nlp)

In [4]:
# Chargement du fichier
with open("transcriptions_ordonnées.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
corpus_correcte = []

for entry in data:
    i = 1
    while f"version correcte en polonais ({i})" in entry:
        phrase = entry[f"version correcte en polonais ({i})"]
        corpus_correcte.append(phrase)
        i += 1

# Lemmatisation et comptage des lemmes (y compris les stopwords)
lemmes = []

for phrase in corpus_correcte:
    doc = nlp(phrase)
    for token in doc:
        if all(c.isalpha() or c == '-' for c in token.text):  # On garde tous les mots alphabétiques, même les stopwords
            lemmes.append(token.lemma_.lower())

# Comptage des fréquences des lemmes
frequence_lemmes = Counter(lemmes)

# Création d'une liste de tuples triée par fréquence décroissante
liste_tuples_correcte = frequence_lemmes.most_common()

# Affichage de la liste complète
print(liste_tuples_correcte)

[('w', 303), ('ulica', 302), ('prosto', 282), ('prosić', 246), ('obok', 226), ('iść', 198), ('i', 192), ('skręcić', 164), ('lewo', 161), ('za', 140), ('dobry', 128), ('sklep', 122), ('prawo', 102), ('na', 102), ('szpital', 101), ('słowacki', 76), ('niski', 73), ('cztery', 68), ('juliusz', 65), ('się', 64), ('być', 62), ('teatr', 54), ('dom', 53), ('szkoła', 52), ('restauracja', 45), ('nazywać', 36), ('numer', 35), ('do', 35), ('znajdować', 28), ('uniwersytet', 27), ('kowalski', 26), ('skręć', 24), ('potem', 19), ('tak', 18), ('park', 14), ('a', 12), ('aż', 10), ('biblioteka', 9), ('lewy', 9), ('tam', 7), ('jechać', 7), ('to', 6), ('róg', 6), ('idź', 5), ('słowackiego', 5), ('skrzyżowano', 5), ('skrzyżowanie', 4), ('tutaj', 3), ('środek', 3), ('zielony', 3), ('stacja', 3), ('uwaga', 2), ('po', 2), ('jedź', 2), ('krótki', 2), ('okej', 2), ('pierwszy', 1), ('dobro', 1), ('nie', 1), ('od', 1), ('przed', 1), ('też', 1), ('trochę', 1), ('bank', 1), ('babcia', 1), ('anna', 1), ('dół', 1), ('m

In [6]:
# Liste pour contenir tous les textes extraits
corpus_entier = []

# Extraction des segments : transcription automatique, manuelle et corrigée
for entry in data:
    # Transcription automatique
    if "transcription automatique" in entry:
        corpus_entier.append(entry["transcription automatique"])

    # Transcriptions manuelles et versions correctes
    i = 1
    while f"transcription manuelle ({i})" in entry or f"version correcte en polonais ({i})" in entry:
        if f"transcription manuelle ({i})" in entry:
            corpus_entier.append(entry[f"transcription manuelle ({i})"])
        if f"version correcte en polonais ({i})" in entry:
            corpus_entier.append(entry[f"version correcte en polonais ({i})"])
        i += 1

# Lemmatisation et comptage de tous les lemmes (y compris les stopwords)
lemmes = []

for texte in corpus_entier:
    doc = nlp(texte)
    for token in doc:
        if all(c.isalpha() or c == '-' for c in token.text):
            lemmes.append(token.lemma_.lower())

# Statistiques de fréquence
frequence_lemmes = Counter(lemmes)

# Résultat : liste de tuples triée
liste_tuples_entier = frequence_lemmes.most_common()

# Affichage
print(liste_tuples_entier)

[('ulica', 702), ('prosić', 682), ('prosto', 673), ('obok', 594), ('i', 561), ('w', 522), ('na', 469), ('iść', 456), ('lewo', 428), ('za', 365), ('skręcić', 347), ('dobry', 304), ('sklep', 285), ('prawo', 260), ('szpital', 200), ('być', 184), ('niski', 159), ('się', 148), ('dom', 143), ('słowacki', 139), ('teatr', 124), ('do', 108), ('cztery', 98), ('szkoła', 95), ('juliusz', 93), ('a', 92), ('restauracja', 88), ('nazywać', 85), ('numer', 84), ('kowalski', 60), ('dobro', 56), ('ulicz', 54), ('tak', 53), ('uniwersytet', 52), ('iś', 48), ('znajdować', 43), ('prosty', 42), ('potem', 41), ('park', 40), ('skręć', 37), ('spital', 36), ('po', 30), ('szklep', 30), ('lewy', 27), ('to', 26), ('sztera', 25), ('jechać', 22), ('skręczać', 22), ('aż', 20), ('słowackiego', 19), ('róg', 18), ('ulicon', 18), ('biblioteka', 18), ('z', 16), ('znać', 15), ('szkol', 15), ('o', 15), ('skrencić', 14), ('restoracja', 14), ('stery', 13), ('idź', 12), ('ospital', 12), ('nisko', 12), ('ja', 12), ('kręcić', 12), 

In [7]:
# Création du dictionnaire avec les lemmes du premier corpus
dictionnaire = {}

# Remplissage du dictionnaire avec les lemmes du premier corpus
for lemma, count in liste_tuples_correcte:
    dictionnaire[lemma] = set()

# Extraction des mots du deuxième corpus qui correspondent aux lemmes du premier
for texte in corpus_entier:
    doc = nlp(texte)
    for token in doc:
        if all(c.isalpha() or c == '-' for c in token.text):
            lemma = token.lemma_.lower()
            if lemma in dictionnaire:
                dictionnaire[lemma].add(token.text)

# Affichage du dictionnaire final
dic = {lemma: list(words) for lemma, words in dictionnaire.items() if len(words) > 0}
print(dic)

# Affichage des mots dont les lemmes ne figurent pas dans le premier corpus
lemmes_dans_le_second_corpus = set([
    token.lemma_.lower() 
    for texte in corpus_entier 
    for token in nlp(texte) 
    if all(c.isalpha() or c == '-' for c in token.text)
])

lemmes_inexistants = lemmes_dans_le_second_corpus - set(dictionnaire.keys())
mots_inexistants = set()

for texte in corpus_entier:
    doc = nlp(texte)
    for token in doc:
        if token.lemma_.lower() in lemmes_inexistants and all(c.isalpha() or c == '-' for c in token.text):
            mots_inexistants.add(token.text)

# Affichage des mots qui n'ont pas de lemme dans le premier corpus
print("\nMots sans lemme dans le premier corpus : \n")
print(mots_inexistants)


{'w': ['we', 'w'], 'ulica': ['ulic', 'ul', 'ulicy', 'ulicę', 'ulicą', 'ulice', 'ulica'], 'prosto': ['prosto'], 'prosić': ['proszę', 'proszą', 'prosi'], 'obok': ['obok'], 'iść': ['idziemy', 'idź', 'idzie', 'iść', 'szło'], 'i': ['i'], 'skręcić': ['skręcenie', 'skręci', 'skręcić', 'skręcisz'], 'lewo': ['lewo'], 'za': ['za'], 'dobry': ['dobrej', 'dobre', 'dobra', 'dobry', 'dobrą'], 'sklep': ['sklep', 'sklepem', 'sklepie', 'sklepu'], 'prawo': ['prawo'], 'na': ['na'], 'szpital': ['szpital', 'szpitalu', 'szpitalem', 'szpitala', 'szpitale'], 'słowacki': ['słowackie', 'słowackiego', 'słowackiej'], 'niski': ['niski', 'niska', 'niską', 'niskie'], 'cztery': ['cztery'], 'juliusz': ['juliusz', 'juliusze', 'juliusza'], 'się': ['się', 'sie'], 'być': ['są', 'bądź', 'jesteśmy', 'jestem', 'jest'], 'teatr': ['teatr', 'teatrze', 'teatrem', 'teatru', 'teatry'], 'dom': ['dom', 'domu'], 'szkoła': ['szkole', 'szkołę', 'szkoły', 'szkoła'], 'restauracja': ['restauracji', 'restauracja', 'restauracje', 'restauracj

In [8]:
lemmatized_dict = {}

# Comparaison et attribution des mots du second corpus aux lemmes les plus proches du premier corpus
for mot in mots_inexistants:
    min_distance = float('inf')
    best_match = None
    # Comparer chaque mot avec les lemmes du premier corpus
    for lemma, lemmes in dic.items():
        for lemma_version in lemmes:
            distance = Levenshtein.distance(mot, lemma_version)
            if distance < min_distance:
                min_distance = distance
                best_match = lemma
    # alignement mot -> lemme
    if best_match:
        if best_match in lemmatized_dict:
            lemmatized_dict[best_match].append(mot)
        else:
            lemmatized_dict[best_match] = [mot]

print(lemmatized_dict)

{'prosić': ['prożę', 'proś', 'piese', 'pros', 'prościej', 'prokaczić', 'muszę', 'slowi', 'pres', 'krakowie', 'prosz', 'przeszłam', 'ospi', 'ploszę', 'przekroczyć', 'prosę', 'prosztę', 'proce', 'stronie', 'poszę'], 'szpital': ['spitala', 'hospitałem', 'ospitalem', 'spitalem', 'spytan', 'oszpitalu', 'zapital', 'spitalu', 'hospitalu', 'szczęśliwa', 'spytal', 'sztabu', 'spital', 'szpitaem', 'szpitalę', 'hospitalem', 'warsztatem', 'zospitalu', 'spita', 'hospital', 'ospital', 'zbytal', 'szpita', 'spitae', 'hospitale', 'oszpitalem', 'oszpital', 'spitale', 'ospitalu'], 'skręć': ['skirzo', 'strejć', 'styryć', 'skrenć'], 'ulica': ['ulico', 'ulidza', 'juliu', 'skul', 'uniwezy', 'julicka', 'alica', 'ulisję', 'ulicjaniska', 'uli', 'skuł', 'olicę', 'elicha', 'żuliesa', 'kol', 'ulisze', 'dżulienca', 'jul', 'olicy', 'uliczka', 'ulcze', 'ulicaniska', 'julicza', 'ulicza', 'uliszia', 'luczi', 'ulicze', 'julisa', 'julia', 'ulisza', 'ulicz', 'ulipia', 'ulisja', 'alika', 'ulicon', 'ulicia', 'huliucka', 'uli

In [9]:
dic_final = {}

# On ajoute tout les mots du premier corpus
for lemma, lemmes in dic.items():
    dic_final[lemma] = lemmes

# On ajoute tout les mots du deuxieme corpus
for lemma, mots in lemmatized_dict.items():
    if lemma in dic_final:
        dic_final[lemma] = list(set(dic_final[lemma] + mots))  # on évite les doublons
    else:
        dic_final[lemma] = mots

# Ajout manuel du mot "4" au lemme "cztery"
if "cztery" in dic_final:
    if "4" not in dic_final["cztery"]:
        dic_final["cztery"].append("4")
else:
    dic_final["cztery"] = ["4"]

print(dic_final)

{'w': ['złe', 'żywę', 'obe', 'iźe', 'ez', 'wy', 'we', 'pre', 'ne', 'wo', 'je', 'del', 'cz', 'le', 'w', 'dze', 'żę', 'z', 'res', 'es', 'tre', 'o', 'e', 'ze'], 'ulica': ['alika', 'uniwezy', 'ulika', 'ulisa', 'ulcze', 'ulicaniska', 'ulico', 'luczi', 'ulicie', 'ulisze', 'dżulienca', 'skuł', 'dżuliska', 'juliu', 'ulicon', 'ulica', 'ulicze', 'ulicja', 'jul', 'allora', 'uliczia', 'ul', 'julisa', 'juli', 'skul', 'olicy', 'juniwe', 'julicka', 'ulicia', 'alica', 'ulisję', 'ulicjaniska', 'olicę', 'uliczka', 'huliucka', 'ulic', 'ulisza', 'ulidze', 'elicha', 'ulicy', 'ulicą', 'uliszia', 'żuliesa', 'ulicz', 'ulice', 'uli', 'julicza', 'slo', 'julia', 'ulicza', 'ulisę', 'ulipia', 'ulicę', 'kol', 'ulidza', 'juliane', 'ulisja', 'dula'], 'prosto': ['proszte', 'proces', 'restora', 'prosta', 'prosteu', 'prochon', 'proszto', 'prosty', 'prastu', 'turystą', 'prusto', 'proto', 'powsta', 'preusto', 'prosto', 'proste', 'prasto', 'prostro', 'prastou', 'prostu', 'prasta', 'presto', 'posto'], 'prosić': ['prożę', 'p

In [10]:
with open("dic_final.json", "w", encoding="utf-8") as f:
    json.dump(dic_final, f, ensure_ascii=False, indent=4)